#### donut-base
- swintransformer + bart-base-decoder
#### donut-large
- swintransformer + bart-larger-decoder

In [1]:
MP1 = "J:/model/mllm-model/donut-pretrain/20240102/pl-checkpoint-159500-ned-0.8607632262639043"
MP2 = "J:/model/facebook_bart-large"

In [10]:
import sys
from transformers import (VisionEncoderDecoderModel,
                          VisionEncoderDecoderConfig,
                          DonutProcessor,
                          AutoProcessor,
                          BartModel,
                          BartConfig,
                          AutoTokenizer)
sys.path.append("../../src/multimodallm")

In [3]:
from multimodallm.utils import cal_model_parameters

In [6]:
donut_base_config = VisionEncoderDecoderConfig.from_pretrained(MP1)
donut_base_model = VisionEncoderDecoderModel.from_pretrained(MP1)

In [18]:
donut_base_params = cal_model_parameters(donut_base_model)
donut_base_encoder_params = cal_model_parameters(donut_base_model.encoder)
donut_base_decoder_params = cal_model_parameters(donut_base_model.decoder)
donut_base_params, donut_base_encoder_params, donut_base_decoder_params

(204299384, 74180728, 130118656)

In [11]:
bart_large_config = BartConfig.from_pretrained(MP2)
bart_large_model = BartModel.from_pretrained(MP2)
bart_large_tokenizer = AutoTokenizer.from_pretrained(MP2)

In [19]:
bart_large_params = cal_model_parameters(bart_large_model)
bart_large_encoder_params = cal_model_parameters(bart_large_model.encoder)
bart_large_decoder_params = cal_model_parameters(bart_large_model.decoder)
bart_large_params, bart_large_encoder_params, bart_large_decoder_params

(406291456, 203678720, 254084096)

In [14]:
inputs = bart_large_tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = bart_large_model(**inputs)
outputs

Seq2SeqModelOutput(last_hidden_state=tensor([[[ 0.5512,  0.8389, -1.4707,  ...,  1.3124, -0.2047,  0.2392],
         [ 0.5512,  0.8389, -1.4707,  ...,  1.3124, -0.2047,  0.2392],
         [ 0.9143,  0.9399, -1.2426,  ...,  0.9183, -0.1838, -0.9975],
         ...,
         [ 0.2561,  0.2253,  0.4470,  ...,  0.3447,  0.0087,  1.5508],
         [ 0.2077, -1.3086, -1.4295,  ..., -0.2998,  0.1828,  0.4700],
         [-0.4893,  2.5148, -1.5513,  ...,  0.5783,  1.0961,  0.1736]]],
       grad_fn=<NativeLayerNormBackward0>), past_key_values=((tensor([[[[ 2.7406e-01,  4.6150e-01,  1.9901e+00,  ...,  9.9268e-01,
            2.8700e+00, -1.3778e+00],
          [-7.4523e-02,  6.4510e-01,  1.7084e+00,  ...,  9.1152e-01,
            2.5264e+00, -1.6802e+00],
          [-2.0921e+00, -1.4773e+00, -3.0118e+00,  ..., -3.9234e+00,
           -4.3515e+00, -2.2497e+00],
          ...,
          [-1.4035e+00, -1.4510e+00, -2.1774e+00,  ...,  4.0009e-02,
           -3.7559e+00,  1.1078e+00],
          [-2.57

### donut-large

In [21]:
donut_large_config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(
    donut_base_model.encoder.config, bart_large_model.decoder.config
)
donut_large_model = VisionEncoderDecoderModel(
    donut_large_config, donut_base_model.encoder, bart_large_model.decoder
)

In [23]:
donut_large_model_params = cal_model_parameters(donut_large_model)
donut_large_model, donut_large_model_params

(VisionEncoderDecoderModel(
   (encoder): DonutSwinModel(
     (embeddings): DonutSwinEmbeddings(
       (patch_embeddings): DonutSwinPatchEmbeddings(
         (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
       )
       (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.0, inplace=False)
     )
     (encoder): DonutSwinEncoder(
       (layers): ModuleList(
         (0): DonutSwinStage(
           (blocks): ModuleList(
             (0-1): 2 x DonutSwinLayer(
               (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
               (attention): DonutSwinAttention(
                 (self): DonutSwinSelfAttention(
                   (query): Linear(in_features=128, out_features=128, bias=True)
                   (key): Linear(in_features=128, out_features=128, bias=True)
                   (value): Linear(in_features=128, out_features=128, bias=True)
                   (dropout): Dropout(p=0.0, inpla

In [24]:
SAVE_MP = "J:/model/mllm-model/donut-large"

In [32]:
processor = DonutProcessor.from_pretrained(MP1)
donut_large_model.config.pad_token_id = processor.tokenizer.pad_token_id
donut_large_model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<s>"])[0]

donut_large_model.config.pad_token_id, donut_large_model.config.decoder_start_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(1, 0)

In [33]:
donut_large_model.save_pretrained(SAVE_MP)
processor.save_pretrained(SAVE_MP)